# Market Data Integration Guide

## Overview
This guide covers integrating with TastyTrade's market data API using Python WebSocket implementation, with a focus on asyncio-based approach.

## API Resources
- [Developer Portal](https://developer.tastytrade.com/)
- [Streaming Market Data Documentation](https://developer.tastytrade.com/streaming-market-data/)
- [DXLink Protocol Reference](https://demo.dxfeed.com/dxlink-ws/debug/#/protocol)

## References
- dxFeed API [doc](https://dxfeed.readthedocs.io/en/stable/) | [github](https://github.com/dxFeed/dxfeed-python-api) | [pypi](https://pypi.org/project/dxfeed/)
- Web Widget Demo [docs](https://tools.dxfeed.com/python-demo/?_ga=2.236532102.669062738.1733058631-563197392.1733058631) | [github](https://github.com/dxFeed/dxfeed-python-api-web-widget-example?tab=readme-ov-file)

## WebSocket Implementation Details

### Selected Library
We'll be using the [websockets](https://websockets.readthedocs.io/en/stable/) library for Python, which provides high-level WebSocket implementation (not to be confused with the lower-level `websocket` library).

### Key Resources
- **Documentation**: Complete guides available on [readthedocs.io](https://websockets.readthedocs.io/en/stable/)
- **Reference Implementation**: [Example using Threading](https://github.com/LordKaT/tastytrade_api_thing/blob/main/lib/TTWebsocket.py) by [LordKaT](https://github.com/LordKaT)

### Technical Approach
The implementation will use `asyncio` due to its comprehensive [feature support](https://websockets.readthedocs.io/en/stable/reference/features.html) in the websockets library.

### AsyncIO Learning Resources
For better understanding of the asyncio implementation:
- [How does async/await work in Python 3.5](https://snarky.ca/how-the-heck-does-async-await-work-in-python-3-5/)
- [Cooperative multitasking with Coroutines](https://pymotw.com/3/asyncio/coroutines.html)
- [A Curious course on Coroutines and Concurrency](http://www.dabeaz.com/coroutines/)


In [ ]:
from tastytrade.logging import setup_logging
import logging
import asyncio
from tastytrade.sessions import Credentials
from tastytrade.sessions.requests import AsyncSessionHandler
from tastytrade.sessions.sockets import WebSocketManager
from tastytrade.sessions.messaging import Channels
from tastytrade.sessions.dxlink import DXLinkClient
from decimal import Decimal
from datetime import datetime, timedelta

logging.getLogger().handlers.clear()

TEST = True
ENV = "Live"

shutdown = asyncio.Event()


setup_logging(
    level=logging.DEBUG,
    # log_dir="/var/log/tastytrade",
    log_dir="tastytrade",
    filename_prefix=f"{'dev' if TEST else 'prod'}_tastytrade",
    console=True,
    file=True,
)


def get_trade_day() -> str:
    trade_day = datetime.now()
    while trade_day.weekday() >= 5:
        trade_day += timedelta(days=1)

    return trade_day.strftime("%y%m%d")


def round_decimal(value, precision=5) -> float:
    target = str(precision)

    if not isinstance(value, Decimal):
        value = Decimal(value)

    if precision > 0:
        result = (value / Decimal(target)).quantize(
            Decimal("1"), rounding="ROUND_HALF_UP"
        ) * Decimal(target)
        return float(result)

    else:
        return float(round(value, abs(precision)))

## Test WebSocket context manager

In [ ]:
"""
TODO 

  1. Find nearest ATM strike
  2. list of symbols
      +-10 pts from ATM strike
      both Calls & Puts
"""

session = await AsyncSessionHandler.create(Credentials(env=ENV))

yy_mm_dd = get_trade_day()

symbols = [
    "SPX",
    "NVDA",
    "BTC/USD:CXTALP",
    f".SPXW{yy_mm_dd}P5990",
    f".SPXW{yy_mm_dd}P5995",
    f".SPXW{yy_mm_dd}P6000",
]

async with WebSocketManager(session) as websocket:
    dxlink_client = DXLinkClient(websocket)
    await dxlink_client.setup_feeds()
    await dxlink_client.subscribe_to_feeds(symbols)

    if TEST:
        await asyncio.sleep(10)
    else:
        await websocket.send_keepalives()

    await dxlink_client.queue_manager.cleanup()

await session.close()

## Test individual components

In [ ]:
session = await AsyncSessionHandler.create(Credentials(env=ENV))

exlink = WebSocketManager(session)
await exlink.open()

In [ ]:
dxlink_client = DXLinkClient(exlink)
await dxlink_client.setup_feeds()
await dxlink_client.subscribe_to_feeds(["SPX"])

In [ ]:
asyncio.all_tasks()

In [ ]:
await exlink.close()

## Old Feed Data

In [ ]:
# session = await AsyncSessionHandler.create(Credentials(env=ENV))

In [ ]:
async def get_quote(symbol: str, type: str = "mid") -> dict:
    url = f"https://api.tastyworks.com/market-data/quotes/{symbol}"

    async with session.session.get(url, headers=session.session.headers) as response:
        response_data = await response.json()

    return response_data.get("data", {}).get(type)


quote = await get_quote("SPX")
print(f"SPX{get_trade_day()}{round_decimal(quote):.0f}")

In [ ]:
vars(exlink)

In [ ]:
getattr(exlink, "none")